In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import urllib2
#import sys

In [2]:
#URL넣어주면 해당 박스스코어 파싱
class BoxScoreParser:
    def __init__(self,url):
        self.data = urllib2.urlopen(url)
        self.html = BeautifulSoup(self.data)

In [3]:
data = urllib2.urlopen('http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=1&gameId=20160327NCWO0&gyear=2016')
html = BeautifulSoup(data)

C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
html

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n<head><link href="//image.koreabaseball.com/client/images/common/favicon.ico" rel="shortcut icon"/><meta content="text/html; charset=unicode-escape" http-equiv="content-type"/><meta content="kbo" name="keywords"/><meta content="20130308" name="author-date"/><meta content="IE=9, Chrome" http-equiv="x-ua-compatible"/><title>\r\n\t\ubc15\uc2a4\uc2a4\ucf54\uc5b4 | \uacbd\uae30\uc77c\uc815/\uacb0\uacfc | KBO\r\n</title><link href="//image.koreabaseball.com/client/css/reset.css?version=201503111" rel="stylesheet" type="text/css"/><link href="//image.koreabaseball.com/client/css/common.css?version=20160310" rel="stylesheet" type="text/css"/><link href="//image.koreabaseball.com/client/css/jquery-ui.css" rel="stylesheet" type="text/css"/><link href="//image.koreabaseball.com/client/css/jquery.bxslider.css?version=20151006" rel="

In [7]:
#DATE
date=html.select('div.yearDate span')[0].text

#AWAY팀명
a_Team=html.select('table.socreBoard tr:nth-of-type(2) th')[0].text

#HOME팀명
h_Team=html.select('table.socreBoard tr:nth-of-type(3) th')[0].text

#AWAY_SCORE
tmp=html.select('table.socreBoard tr:nth-of-type(2) td')
a_Score=list()
for score in tmp:
    a_Score.append(score.text)

#HOME_SCORE
tmp=html.select('table.socreBoard tr:nth-of-type(3) td')
h_Score=list()
for score in tmp:
    h_Score.append(score.text)

#STADIUM
tmp=html.select('p.ballpark')[0]
#.text 메서드는 unicode형태로 반환
tmp=tmp.text
stadium=tmp[tmp.find(u'고척'):tmp.find(u'관중')].strip()

#AWAY 투수기록
col=[u'선수명',u'등판',u'결과',u'승',u'패',u'세',u'이닝',u'타자',u'투구수',u'타수',u'피안타',u'홈런',u'4사구',u'삼진',u'실점',u'자책',u'평균자책점']
frame=html.select('table#xtable3 tbody:nth-of-type(1) tr')
data=list()    #frame으로부터 데이터를 담을 리스트
for row in frame:
    tmp=list()
    for dat in row.text.strip().split('\n'):    #한 행(한 투수의 데이터)마다 split한뒤 list로 정리
        tmp.append(dat)
    data.append(tmp)   #정리된 list를 data에 추가
a_pitRecord = pd.DataFrame(data=data,columns=col)

#HOME 투수기록
#col=[u'선수명',u'등판',u'결과',u'승',u'패',u'세',u'이닝',u'타자',u'투구수',u'타수',u'피안타',u'홈런',u'4사구',u'삼진',u'실점',u'자책',u'평균자책점']
frame=html.select('table#xtable3 tbody:nth-of-type(2) tr')
data=list()    #frame으로부터 데이터를 담을 리스트
for row in frame:
    tmp=list()
    for dat in row.text.strip().split('\n'):    #한 행(한 투수의 데이터)마다 split한뒤 list로 정리
        tmp.append(dat)
    data.append(tmp)   #정리된 list를 data에 추가
h_pitRecord = pd.DataFrame(data=data,columns=col)



In [6]:
a_pitRecord

,선수명,등판,결과,승,패,세,이닝,타자,투구수,타수,피안타,홈런,4사구,삼진,실점,자책,평균자책점
0,이재학,선발,승,2,2,0,4,18,63,16,5,1,2,6,3,3,4.58
1,구창모,5.1,,0,0,0,1,4,18,3,0,0,1,0,0,0,4.91
2,김선규,6.5,,0,0,0,1,3,10,3,1,0,0,1,0,0,6.75
3,강장산,7.8,,0,0,0,1,3,11,3,1,0,0,0,0,0,3.86
4,박준영,8.2,,0,0,0,1,3,11,3,1,0,0,0,0,0,1.69
5,박민석,9.5,,1,0,1,1,3,6,3,0,0,0,0,0,0,0.00


In [8]:
h_pitRecord

,선수명,등판,결과,승,패,세,이닝,타자,투구수,타수,피안타,홈런,4사구,삼진,실점,자책,평균자책점
0,박주현,선발,패,0,1,0,2 2/3,18,63,18,11,1,0,3,7,7,7.27
1,김상수,3.1,,1,1,0,3 1/3,13,57,12,3,0,1,3,0,0,1.38
2,마정길,7.5,,0,0,0,1,4,9,4,1,1,0,0,1,1,2.57
3,김택형,8.9,,0,0,0,1,3,14,3,0,0,0,0,0,0,0.00
4,이보근,9.3,,0,0,0,1,5,19,5,2,0,0,1,0,0,1.80


In [77]:
#등판:선발의 선수명과 이닝,타자를 get
print h_pitRecord.ix[h_pitRecord.ix[u'등판']][u'선수명':u'타자']
#[u'선수명',u'이닝',u'타자']

KeyError: u'\ub4f1\ud310'

In [101]:
a=h_pitRecord.select(lambda x: x==u'등판',axis=1)

In [102]:
a

,등판
0,선발
1,3.1
2,7.5
3,8.9
4,9.3


In [125]:
a[a[u'등판']=='7.5'].index

Int64Index([2], dtype='int64')

In [126]:
a[u'등판']=='7.5'

0    False
1    False
2     True
3    False
4    False
Name: 등판, dtype: bool

In [10]:
a_pitRecord

,선수명,등판,결과,승,패,세,이닝,타자,선수명,타수,피안타,홈런,4사구,삼진,실점,자책,평균자책점


In [ ]:
a_pitRecord.ge

In [8]:
print date
print a_Team
print h_Team
print a_Score
print h_Score
print stadium
print len(stadium)

2016.03.27(일)
NC
넥센
[u'1', u'3', u'3', u'0', u'0', u'0', u'1', u'0', u'0', u'-', u'-', u'-', u'8', u'17', u'0', u'1']
[u'0', u'2', u'1', u'0', u'0', u'0', u'0', u'0', u'0', u'-', u'-', u'-', u'3', u'8', u'1', u'3']
고척
2


In [140]:
print len(u'고현영')
print len('고현영')

3
9


In [ ]:
print tmp.find(u'고척')

In [126]:
print stadium


        구장 : 고척      
        관중 : 3,476      
        개시 : 13:00      
        종료 : 15:44      
        경기시간 : 2:44
    
